# INFO F422 - Statistical Fundation of Machine Learning
## Project "House Prices : Advanced Regression Techniques"

    Erica Berghman
    Master 1 - Brussels Engineer School

## Abstract

## 1. Introduction 

* with dataset description, goals, and an overview of the report structure 

Starting from a data set with 81 criteria about houses and their selling price, the goal is to create a model capable of predicting the price of other houses given some of these criterias. A good model description is a model that has been refined multiple types. This report will show the methodology used to construct a model for this particular problem. It is based on the methodology of the Chapter 6 of the syllabus.

## 2. Preprocessing 

In [126]:
dataSample = 400
mean = T          # variable to determine if we use the mean or the median to replace the NA values
set.seed(1)

source("functions/replaceNA.R")
# Hide warnings
options(warn=-1)

### 2.1 Preprocessing the data set

In order to get a model, the data must be preprocessed. Firstly we read the data given and we take a sample set of 400 houses out of the 1460. There is 81 criteria.

In [127]:
data<-read.csv("input/train.csv")
data.sample<-data[sample(nrow(data),dataSample),]
#dim(data.sample)
#data[1:2,]

The categorical (factor) criterias are removed.

In [128]:
factor_variables<-which(sapply(data.sample[1,],class)=="factor")
data.sample.nofactor<-data.sample[,-factor_variables]
data.sample.factor<-data.sample[,factor_variables]
#summary(data.sample.factor)

In [129]:
library(dummies)
variable_to_keep<-c("CentralAir", "Street", "LotShape")
data_factor_onehot <- dummy.data.frame(data.sample.factor[,variable_to_keep], sep="_")
data.nofactor.extended<-cbind(data.sample.nofactor,data_factor_onehot)

#### Missing data 
The missing values (NA) are replaced by an estimator of these values (eg. mean or median).Ca

In [130]:
if (mean) {
    data_preprocessed<-data.frame(apply(data.nofactor.extended,2,replace_na_with_mean_value)) 
} else {
    data_preprocessed<-data.frame(apply(data.nofactor.extended,2,replace_na_with_median_value))
}

## 3. Feature selection 
Methodology and main results

The text must contain the list of selected variables and the motivation of their choice. The use of formulas, tables and pseudo-code to describe the feature selection procedure is encouraged. 

#### Redundant and irrelevant features 

The "Id" column which is irrelevant is deleted.

In [131]:
data_preprocessed<-data_preprocessed[,setdiff(colnames(data_preprocessed),"Id")]

The criterias that are redundant (linear combination of others criterias and correlation > 0.99) are deleted.

In [132]:
library(caret)
library(ggplot2)
library(lattice)

linearCombo.idx <- findLinearCombos(data_preprocessed)$remove
if (!is.null(linearCombo.idx)) data_preprocessed<-data_preprocessed[,-linearCombo.idx]

correlation.matrix <- cor(data_preprocessed)
correlation.matrix[upper.tri(correlation.matrix)] <- 0
diag(correlation.matrix) <- 0
data.uncorrelated <- data_preprocessed[,!apply(correlation.matrix,2,function(x) any(abs(x) > 0.99))]

The input and output vectors are created and scaled.

In [133]:
X <- data.uncorrelated[,setdiff(colnames(data.uncorrelated),"SalePrice")]
Y <- data.uncorrelated[,"SalePrice"]
X <- data.frame(X)
#Y <- data.frame(Y)
X.scale <- data.frame(scale(X))
Y.scale <- scale(Y)

N<-nrow(X)    #Number of examples
n<-ncol(X)    #Number of input variables

In [143]:
dim(X)

[1] 400  39

Two feature selection methods are implemented in the featureSelection file:

** 1. Filter method using correlation with the variable to determine.**

   It create a subset of features, removing from the whole features set the ones less likely to determine the variable (SalePrice). It is robust to overfitting and effective in computational time. However it might select redundant variables as the interraction between the variables is not taken in consideration.  
   
** 2. Wrapper method**

   Its a cyclic method where a subset of variable is created and evaluated by the Learning Algorithm, modifying the chosen subset. This is done until the best subset is generated.  
    
The filter method is used to select a first "big" set of features, that is then refined by the wrapper method. This gives us the possibility use advantages of both method to get a good subset in a relatively correct computational time.

In [135]:
source("functions/filtre.R")
features.filtre <- filtre(X.scale,Y.scale)  # return the idx of the more correlated features where #feature = argmin(CV error)

In [136]:
X.filtre = X.scale[,features.filtre]
dim(X.filtre)
X.filtre[1:2,]

[1] 400  33

,OverallQual,X1stFlrSF,GarageCars,GarageArea,FullBath,YearRemodAdd,TotRmsAbvGrd,YearBuilt,Fireplaces,MasVnrArea,...,MSSubClass,BedroomAbvGr,KitchenAbvGr,EnclosedPorch,ScreenPorch,X3SsnPorch,MiscVal,OverallCond,LotShape_IR2,BsmtHalfBath
388,-0.09423743,-0.108574,-0.9763774,-0.5683105,-1.0707010,-0.4480077,-0.3212315,0.1227521,0.5921614,0.7718430,...,0.6140872,0.1035,-0.2291288,-0.3559642,-0.2798042,-0.117066,-0.1506729,0.3556585,-0.1599279,-0.2291288
543,0.59112569,1.346596,0.2753885,0.4394986,0.7595571,0.6441150,0.2906380,0.8516444,0.5921614,-0.5959218,...,-0.8436866,0.1035,-0.2291288,-0.3559642,-0.2798042,-0.117066,-0.1506729,-0.4811850,-0.1599279,-0.2291288


In [137]:
source("functions/mRMR.R")
features.mrmr <- mrmr(X.scale, Y.scale)    # return the idx of the more correlated features where #feature = argmin(CV error)

In [138]:
X.mrmr <- X.scale[,features.mrmr]
dim(X.mrmr)

[1] 400  35

In [139]:
source("functions/pca.R")
X.pca <- pca(X.scale, Y.scale)   # return X_pca with nb of columns = argmin(CV error)
dim(X.pca)

[1] 400  39

In [140]:
source("functions/wrapper.R")
features.wrapper <- wrapper(X.scale, Y.scale)

In [141]:
X.wrapper <- X[,features.wrapper]
dim(X.wrapper)

[1] 400  39

In [142]:
features.wrapper.pca <- wrapper(X.pca, Y.scale)


[1] "MSSubClass"    "LotArea"       "BsmtHalfBath"  "GarageArea"   
 [5] "LotShape_IR2"  "Fireplaces"    "GarageCars"    "X1stFlrSF"    
 [9] "PoolArea"      "BsmtFinSF2"    "GarageYrBlt"   "MiscVal"      
[13] "CentralAir_Y"  "MasVnrArea"    "YearRemodAdd"  "WoodDeckSF"   
[17] "LotShape_IR3"  "YrSold"        "EnclosedPorch" "FullBath"     
[21] "HalfBath"      "X3SsnPorch"    "Street_Grvl"   "MoSold"       
[25] "X2ndFlrSF"     "OpenPorchSF"   "BsmtUnfSF"     "LowQualFinSF" 
[29] "LotShape_IR1"  "BedroomAbvGr"  "TotRmsAbvGrd"  "BsmtFullBath" 
[33] "KitchenAbvGr"  "OverallQual"   "BsmtFinSF1"    "LotFrontage"  
[37] "OverallCond"   "ScreenPorch"   "YearBuilt"

## Model selection  
Methodology and main results

For the learning method, the only packages that may be used are those seen during the exercise classes : stats, nnet, tree, lazy, and e1071, for linear models, neural networks, decision trees, nearest neighbours and SVM, respectively.

The accuracy of the regression models during the selection process should be assessed by using the root mean squared error between the logarithm of the
predicted value and the logarithm of the observed sale price.

The text must mention the different
(and at least three) models which have been taken into consideration and the procedure used for model assessment and selection. The use of formulas,
tables and pseudo-code to describe the feature selection procedure is encouraged

In [ ]:
model <- neuralnetwork(X, Y)

## Ensemble techniques : Combination of models strategy
Methodology and main results

The text should mention the different models taken into consideration as well as the techniques used for the combination.

## Discussion and conclusion: 
Summary of your work, and discussion of what worked well, not well, why, what insights you got from the analyses you made. 